In [ ]:
!pip install flask pyngrok flask-cors transformers torch sentencepiece openai-whisper gTTS pydub

from flask import Flask, request, jsonify, make_response, send_file
from flask_cors import CORS
from pyngrok import ngrok
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json
import os
import whisper
from gtts import gTTS
from io import BytesIO
import tempfile

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# ===== Global Variables =====
id2label = {}  # Initialize empty dictionary

# ===== Set Paths =====
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks"
MODEL_PATH = f"{DRIVE_PATH}/saved_model"
LABELS_PATH = f"{MODEL_PATH}/intent_labels.json"

# ===== Verify Paths =====
print("="*40)
print("Verifying paths...")
print("="*40)
print(f"Model path exists: {os.path.exists(MODEL_PATH)}")
print(f"Labels path exists: {os.path.exists(LABELS_PATH)}")

# ===== Load Labels First =====
print("\n" + "="*40)
print("Loading intent labels...")
print("="*40)
try:
    with open(LABELS_PATH, "r") as f:
        label_names = json.load(f)
    id2label = {i: name for i, name in enumerate(label_names)}
    print(f"✅ Loaded {len(id2label)} intent labels")
    print("Sample labels:")
    for i in range(min(5, len(id2label))):
        print(f"  {i}: {id2label[i]}")
except Exception as e:
    print(f"❌ Error loading labels: {str(e)}")

# ===== Load Classifier Model =====
print("\n" + "="*40)
print("Loading classifier model...")
print("="*40)
classifier = None
try:
    classifier = pipeline(
        "text-classification",
        model=MODEL_PATH,
        tokenizer=MODEL_PATH,
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Model loaded successfully")

    # Test classification
    test_query = "How do I reset my password?"
    print(f"\nTesting with query: '{test_query}'")
    test_result = classifier(test_query)[0]
    label_str = test_result["label"]
    label_index = int(label_str.replace("LABEL_", "")) if label_str.startswith("LABEL_") else int(label_str)
    intent = id2label.get(label_index, f"UNKNOWN_LABEL_{label_index}")
    print(f"  Intent: {intent}")
    print(f"  Confidence: {test_result['score']:.4f}")


except Exception as e:
    print(f"❌ Error loading model: {str(e)}")
    classifier = None

# ===== Load Translation Model =====
print("\n" + "="*40)
print("Loading translation model...")
print("="*40)
translator = None
try:
    translator = pipeline(
        "translation",
        model="facebook/nllb-200-distilled-600M",
        device=0 if torch.cuda.is_available() else -1
    )
    print("✅ Translation pipeline ready")

    # Test translation
    test_text = "Hello world"
    test_target = "tel_Telu"
    test_result = translator(test_text, src_lang="eng_Latn", tgt_lang=test_target, max_length=200)
    print(f"🧪 Test translation: '{test_text}' → '{test_result[0]['translation_text']}'")

except Exception as e:
    print(f"❌ Error loading translator: {str(e)}")
    try:
        translator = pipeline(
            "translation",
            model="Helsinki-NLP/opus-mt-en-mul",
            device=0 if torch.cuda.is_available() else -1
        )
        print("✅ Fallback translation model loaded")
    except Exception as fallback_e:
        print(f"❌ Failed to load fallback translator: {str(fallback_e)}")

# ===== Load Whisper Model =====
print("\n" + "="*40)
print("Loading Whisper model...")
print("="*40)
whisper_model = None
try:
    whisper_model = whisper.load_model("base")
    print("✅ Whisper model loaded")
except Exception as e:
    print(f"❌ Error loading Whisper: {str(e)}")

# ===== Ngrok Setup =====
print("\n" + "="*40)
print("Configuring ngrok...")
print("="*40)
NGROK_AUTH_TOKEN = "YOUR_VALID_NGROK_AUTH_TOKEN"  # Replace with your token

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_VALID_NGROK_AUTH_TOKEN":
    try:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        public_url = ngrok.connect(5000).public_url
        print(f"🌐 Public URL: {public_url}")
    except Exception as e:
        print(f"❌ Ngrok error: {str(e)}")
        public_url = None
else:
    print("❌ Ngrok not configured - missing auth token")
    public_url = None


@app.route("/transcribe", methods=["POST"])
def transcribe_audio():
    if not whisper_model:
        return jsonify({"error": "Whisper model not loaded"}), 500

    if 'audio' not in request.files:
        return jsonify({"error": "No audio file provided"}), 400

    try:
        audio_file = request.files['audio']
        lang = request.form.get('lang', 'en')  # default to English if not provided
        print(f"Received audio: {audio_file.filename}, size: {len(audio_file.read())} bytes, lang: {lang}")
        audio_file.seek(0)  # reset file after reading

        with tempfile.NamedTemporaryFile(suffix=".webm", delete=False) as tmp:
            audio_file.save(tmp.name)
            result = whisper_model.transcribe(tmp.name, language=lang)
            os.unlink(tmp.name)

        print("Whisper Result:", result)
        return jsonify({"text": result["text"]})
    except Exception as e:
        print("Transcription Error:", str(e))
        return jsonify({"error": str(e)}), 500

@app.route("/classify", methods=["POST"])
def classify():
    if not classifier:
        return jsonify({"error": "Classifier not loaded"}), 500

    try:
        data = request.json
        user_input = data.get("message", "")
        if not user_input:
            return jsonify({"error": "Empty input"}), 400

        result = classifier(user_input)[0]
        label_str = result["label"]
        label_index = int(label_str.replace("LABEL_", "")) if label_str.startswith("LABEL_") else int(label_str)
        intent = id2label.get(label_index, f"LABEL_{label_index}")

        if "interest" in user_input.lower() and "rate" in user_input.lower():
            if intent == "exchange_rate":
                intent = "interest_rate"

        return jsonify({
            "intent": intent,
            "confidence": round(result["score"], 4),
            "message": user_input
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/translate", methods=["POST"])
def translate():
    if not translator:
        return jsonify({"error": "Translator not loaded"}), 500

    try:
        data = request.json
        text = data.get("text", "")
        target_lang = data.get("target_lang", "tel_Telu")
        if not text:
            return jsonify({"error": "Empty text"}), 400

        if "nllb" in translator.model.name_or_path.lower():
            translated = translator(text, src_lang="eng_Latn", tgt_lang=target_lang, max_length=200)
            translation_text = translated[0]['translation_text']
        else:
            translated = translator(text)
            translation_text = translated[0]['translation_text']

        return jsonify({"translation": translation_text})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/text-to-speech", methods=["POST"])
def text_to_speech():
    try:
        data = request.json
        text = data.get("text", "")
        lang = data.get("lang", "en")
        if not text:
            return jsonify({"error": "Empty text"}), 400

        tts = gTTS(text=text, lang=lang, slow=False)
        mp3_fp = BytesIO()
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)

        return send_file(
            mp3_fp,
            mimetype="audio/mpeg",
            as_attachment=True,
            download_name="output.mp3"
        )
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/health")
def health_check():
    return jsonify({
        "status": "running",
        "classifier_loaded": classifier is not None,
        "translator_loaded": translator is not None,
        "whisper_loaded": whisper_model is not None,
        "labels_loaded": len(id2label) > 0
    })

if __name__ == "__main__":
    print("\n" + "="*40)
    print("Starting Flask server...")
    print("="*40)
    if public_url:
        print(f"🌐 Public URL: {public_url}")
    app.run(port=5000)